In [1]:
# -*- coding: utf-8 -*-
"""Flan-T5 Empathy + Counsel Training"""

# =====================
# 0. Install deps
# =====================
!pip install -q transformers datasets evaluate accelerate sentencepiece rouge_score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
import evaluate
import torch

# =====================
# 1. Load & Preprocess Datasets
# =====================

import pandas as pd
import ast  # to safely parse the JSON-like conversation strings

# Replace with your actual file paths
empathy_train = "/content/mental_health_train.csv"
empathy_val   = "/content/mental_health_valid.csv"
empathy_test  = "/content/mental_health_test.csv"
counsel_path  = "/content/counsel_train.csv"

# --- Load CSVs ---
df_empathy_train = pd.read_csv(empathy_train)
df_empathy_val   = pd.read_csv(empathy_val)
df_empathy_test  = pd.read_csv(empathy_test)
df_counsel       = pd.read_csv(counsel_path)

# --- Helper: Extract assistant responses from 'conversations' ---
def extract_assistant_responses(conv_str):
    try:
        conv_list = ast.literal_eval(conv_str)  # safely parse string to list
        assistant_resps = [c['content'] for c in conv_list if c['role'] == 'assistant']
        return " ".join(assistant_resps) if assistant_resps else ""
    except:
        return ""

# --- Process empathy datasets ---
df_empathy_train_proc = pd.DataFrame({
    "input_text": "As a caring listener, show empathy for: " + df_empathy_train["situation"].astype(str),
    "target_text": df_empathy_train["conversations"].apply(extract_assistant_responses)
})

df_empathy_val_proc = pd.DataFrame({
    "input_text": "As a caring listener, show empathy for: " + df_empathy_val["situation"].astype(str),
    "target_text": df_empathy_val["conversations"].apply(extract_assistant_responses)
})

df_empathy_test_proc = pd.DataFrame({
    "input_text": "As a caring listener, show empathy for: " + df_empathy_test["situation"].astype(str),
    "target_text": df_empathy_test["conversations"].apply(extract_assistant_responses)
})

# --- Process counsel dataset ---
df_counsel_proc = pd.DataFrame({
    "input_text": "As a mental health counselor, provide therapeutic response to: " + df_counsel["Context"].astype(str),
    "target_text": df_counsel["Response"].astype(str)
})

print("✅ Empathy Train Sample:\n", df_empathy_train_proc.head(2))
print("✅ Counsel Sample:\n", df_counsel_proc.head(2))



✅ Empathy Train Sample:
                                           input_text target_text
0  As a caring listener, show empathy for: I reme...            
1  As a caring listener, show empathy for:  i use...            
✅ Counsel Sample:
                                           input_text  \
0  As a mental health counselor, provide therapeu...   
1  As a mental health counselor, provide therapeu...   

                                         target_text  
0  If everyone thinks you're worthless, then mayb...  
1  Hello, and thank you for your question and see...  


In [4]:
# =====================
# 2. Merge into splits
# =====================
# Convert each to HF Dataset
ds_empathy = DatasetDict({
    "train": Dataset.from_pandas(df_empathy_train_proc),
    "validation": Dataset.from_pandas(df_empathy_val_proc),
    "test": Dataset.from_pandas(df_empathy_test_proc),
})

ds_counsel_full = Dataset.from_pandas(df_counsel_proc)
# split counsel into 80/10/10
ds_counsel = ds_counsel_full.train_test_split(test_size=0.2, seed=42)
ds_counsel_valtest = ds_counsel["test"].train_test_split(test_size=0.5, seed=42)

ds_counsel = DatasetDict({
    "train": ds_counsel["train"],
    "validation": ds_counsel_valtest["train"],
    "test": ds_counsel_valtest["test"],
})

# Concatenate empathy + counsel
from datasets import concatenate_datasets
dataset = DatasetDict({
    "train": concatenate_datasets([ds_empathy["train"], ds_counsel["train"]]),
    "validation": concatenate_datasets([ds_empathy["validation"], ds_counsel["validation"]]),
    "test": concatenate_datasets([ds_empathy["test"], ds_counsel["test"]]),
})

print(dataset)

# =====================
# 3. Load tokenizer/model
# =====================
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# =====================
# 4. Tokenization
# =====================
max_input_len = 384  # Increased from 384
max_target_len = 192 # Increased from 192

def preprocess(batch):
    inputs = tokenizer(batch["input_text"], max_length=max_input_len, truncation=True, padding="max_length")
    targets = tokenizer(batch["target_text"], max_length=max_target_len, truncation=True, padding="max_length")
    inputs["labels"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in labels]
        for labels in targets["input_ids"]
    ]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)

# =====================
# 5. Data collator
# =====================
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# =====================
# 6. Metrics (ROUGE) - Safe Version
# =====================
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple):
        preds = preds[0]

    vocab_size = tokenizer.vocab_size

    # Clamp predictions to valid range to prevent OverflowError
    safe_preds = [
        [min(max(int(p), 0), vocab_size - 1) for p in pred] for pred in preds
    ]
    decoded_preds = tokenizer.batch_decode(safe_preds, skip_special_tokens=True)

    # Clamp labels and replace -100 with pad_token_id
    safe_labels = [
        [min(max(int(l) if l != -100 else tokenizer.pad_token_id, 0), vocab_size - 1) for l in label]
        for label in labels
    ]
    decoded_labels = tokenizer.batch_decode(safe_labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Round results for readability
    return {k: round(v, 4) for k, v in result.items()}

# =====================
# 7. Training args
# =====================
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan_t5_empathy_counsel",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,            # more stable than 3e-4
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4, # effective batch size = 16
    num_train_epochs=5,            # longer for better ROUGE
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=100,  # Increased from 100
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    report_to="none"
)

# =====================
# 8. Trainer
# =====================
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# =====================
# 9. Train & Evaluate
# =====================
trainer.train()

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 900
    })
    validation: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 550
    })
    test: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 550
    })
})


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/550 [00:00<?, ? examples/s]

Map:   0%|          | 0/550 [00:00<?, ? examples/s]

/tmp/ipython-input-1102706304.py:117: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,nan,0.007100,0.000400,0.005200,0.005200


KeyboardInterrupt: 

In [ ]:
metrics = trainer.evaluate(tokenized_dataset["test"])
print("Final Test Metrics:", metrics)

# =====================
# 10. Save
# =====================
trainer.save_model("./flan_t5_empathy_counsel_final")
tokenizer.save_pretrained("./flan_t5_empathy_counsel_final")

In [ ]:
# -*- coding: utf-8 -*-
"""Flan-T5 Empathy + Counsel Training with Logging"""

import sys, logging

# =====================
# Setup Logging
# =====================
log_file = "training_log.txt"

# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# File handler
fh = logging.FileHandler(log_file, mode="w")
fh.setLevel(logging.INFO)

# Stream handler (to still see in Colab output)
sh = logging.StreamHandler(sys.stdout)
sh.setLevel(logging.INFO)

# Formatter
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
fh.setFormatter(formatter)
sh.setFormatter(formatter)

# Add handlers
if logger.hasHandlers():
    logger.handlers.clear()
logger.addHandler(fh)
logger.addHandler(sh)

# Redirect print() → logging.info()
print = lambda *args, **kwargs: logging.info(" ".join(map(str, args)))
